In [ ]:
#default_exp model_documentation

# Model documentation
> Functions to jumpstat and facilitate model documentation

In [ ]:
#hide
from nbdev.showdoc import *

Each user has a specific documentation need, ranging from the simply logging the model training to a more complex description of the model pipeline with a discusson of its predictions. `gingado` addresses this variety of needs by offering a class of objects that facilitate model documentation in a generic way, as well as one specific model documentation type, described below. 

The model documentation is performed by Documenters, objects that are analogous to `scikit-learn`'s Estimator, Predictor, Transformer and Model objects (described [here](https://scikit-learn.org/dev/developers/develop.html)). Users with documentation needs beyond the out-of-the-box solutions provided by `gingado` can simply create their own class of Documenters, and compatibility with these custom documentation routines with the rest of the code is ensured.

The baseline `ggdModelDocumentation` class offers code that can be used by any Documenter object to read the pipeline in question and to save the resulting documentation in a variety of formats (eg, HTML, PDF).

Users are encouraged to submit a PR with their own documentation models subclassing `ggdModelDocumentation`.

**Warning: development of documentation is still on-going. Please do not use it yet.**

In [ ]:
#export
import json

class ggdModelDocumentation:
    
    def documentation_path(self):
        pass

    def show_json(self):
        return self.json_doc

    def save_json(self, file_path):
        with open(file_path, 'w') as f:
            json.dump(self.json_doc, f)

    def read_json(self, file_path):
        f = open(file_path)
        self.json_doc = json.load(f)

    def open_questions(self):
        return {k: v for k, v in self.json_doc.items() if v is None}

    def fill_info(self, dict):
        for k, v in dict.items():
            self.json_doc[k] = v

    def __setitem__(self, key, value):
        setattr(self, key, value)

    def __getitem__(self, key):
        return getattr(self, key)

In [ ]:
#hide
#export
from gingado.utils import get_username, get_datetime

In [ ]:
#export
class ModelCard(ggdModelDocumentation):
    def __init__(self, from_template = True):
        if from_template:
            self.start_from_template()

    def start_from_template(self):
        """Creates an empty model card template, then fills it with information that is automatically obtained from the system"""
        self.json_doc = {}
        self.json_doc['model_details'] = {}
        self.json_doc['model_details']['developer'] = get_username()
        self.json_doc['model_details']['datetime'] = get_datetime()
        self.json_doc['model_details']['version'] = "1" 
        self.json_doc['model_details']['task'] = None
        self.json_doc['model_details']['framework'] = None
        self.json_doc['model_details']['architecture'] = None
        self.json_doc['model_details']['parameters'] = None
        self.json_doc['model_details']['optimizer'] = None
        self.json_doc['model_details']['type'] = None
        self.json_doc['model_details']['info'] = None
        self.json_doc['model_details']['citation'] = None
        self.json_doc['model_details']['license'] = None
        self.json_doc['model_details']['contact'] = None

        self.json_doc['intended_use'] = {}
        self.json_doc['intended_use']['primary_uses'] = None
        self.json_doc['intended_use']['primary_users'] = None
        self.json_doc['intended_use']['out_of_scope'] = None

        self.json_doc['factors'] = {}
        self.json_doc['factors']['relevant'] = None
        self.json_doc['factors']['evaluation'] = None

        self.json_doc['metrics'] = {}
        self.json_doc['metrics']['performance_measures'] = None
        self.json_doc['metrics']['thresholds'] = None
        self.json_doc['metrics']['variation_approaches'] = None

        self.json_doc['data'] = {}
        self.json_doc['data']['datasets'] = None
        self.json_doc['data']['preprocessing'] = None
        self.json_doc['data']['motivation'] = None

        self.json_doc['ethical_cons'] = {}
        self.json_doc['ethical_cons']['impact'] = None
        self.json_doc['ethical_cons']['risks'] = None
        self.json_doc['ethical_cons']['mitigation'] = None
        self.json_doc['ethical_cons']['avoid'] = None

        self.json_doc['caveats_recomm'] = {}
        self.json_doc['caveats_recomm']['caveats'] = None
        self.json_doc['caveats_recomm']['recommendations'] = None
        self.json_doc['caveats_recomm']['misc_notes'] = None

        return self  

In [ ]:
show_doc(ModelCard.__init__)

<h4 id="ModelCard.__init__" class="doc_header"><code>ModelCard.__init__</code><a href="__main__.py#L3" class="source_link" style="float:right">[source]</a></h4>

> <code>ModelCard.__init__</code>(**`from_template`**=*`True`*)

Initialize self.  See help(type(self)) for accurate signature.

In [ ]:
show_doc(ModelCard.start_from_template)

<h4 id="ModelCard.start_from_template" class="doc_header"><code>ModelCard.start_from_template</code><a href="__main__.py#L7" class="source_link" style="float:right">[source]</a></h4>

> <code>ModelCard.start_from_template</code>()

Creates an empty model card template, then fills it with information that is automatically obtained from the system

In [ ]:
mdoc = ModelCard()

In [ ]:
mdoc.open_questions()

{}